In [ ]:
import pandas as pd

# Load datasets
fraud_df   = pd.read_csv("../data/raw/Fraud_Data.csv",
                         parse_dates=['signup_time', 'purchase_time'],
                         dtype={'user_id': 'int64',
                                'device_id': 'string',
                                'source': 'string',
                                'browser': 'string',
                                'sex': 'category',    # saves memory
                                'age': 'Int64',       # allows NA
                                'ip_address': 'float'})   # will coerce to int later

credit_df  = pd.read_csv("../data/raw/creditcard.csv")
ip_df      = pd.read_csv("../data/raw/IpAddress_to_Country.csv",
                         dtype={'lower_bound_ip_address': 'int64',
                                'upper_bound_ip_address': 'int64',
                                'country': 'string'})


# Check for missing values
print(fraud_df.isnull().sum())
print(ip_df.isnull().sum())
print(credit_df.isnull().sum())


In [45]:
# check 
print(fraud_df['age'].isnull().any())  

False


In [46]:
# Remove duplicates
ip_df.drop_duplicates(inplace=True)
fraud_df.drop_duplicates(inplace=True)
credit_df.drop_duplicates(inplace=True)

In [ ]:
# Convert time columns to datetime
fraud_df['signup_time'] = pd.to_datetime(fraud_df['signup_time'])
fraud_df['purchase_time'] = pd.to_datetime(fraud_df['purchase_time'])

# Check data types
print(fraud_df.dtypes)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

age_counts = fraud_df.groupby('age')['user_id'].count()
age_counts.plot(kind='bar', figsize=(12, 5))
plt.title("Number of Users by Age")
plt.xlabel("Age")
plt.ylabel("Number of Users")
plt.show()

sns.histplot(fraud_df['age'], kde=True)
plt.title("Age Distribution")
plt.show()

fraud_df['class'].replace({0: 'Legit', 1: 'Fraud'}).value_counts().plot(kind='bar')
plt.title("Transaction Class Distribution")
plt.ylabel("Count")
plt.xlabel("Transaction Type")
plt.show()


In [ ]:
# Boxplot: Purchase Value by Fraud Class with readable labels
sns.boxplot(
    x=fraud_df['class'].map({0: 'Legit', 1: 'Fraudulent'}),
    y=fraud_df['purchase_value']
)
plt.title("Purchase Value by Fraud Class")
plt.xlabel("Transaction Class")
plt.ylabel("Purchase Value")
plt.show()

# Countplot: Source by Class with readable labels
sns.countplot(
    x='source',
    hue=fraud_df['class'].map({0: 'Legit', 1: 'Fraudulent'}),
    data=fraud_df
)
plt.title("Transaction Source by Class")
plt.xlabel("Source")
plt.ylabel("Count")
plt.legend(title='Class')
plt.show()


# sns.boxplot(x='class', y='purchase_value', data=fraud_df)
# plt.title("Purchase Value by Fraud Class")
# plt.show()

# sns.countplot(x='source', hue='class', data=fraud_df)
# plt.title("Source by Class")
# plt.show()

In [50]:
fraud_df['ip_address'].head(10)

0    7.327584e+08
1    3.503114e+08
2    2.621474e+09
3    3.840542e+09
4    4.155831e+08
5    2.809315e+09
6    3.987484e+09
7    1.692459e+09
8    3.719094e+09
9    3.416747e+08
Name: ip_address, dtype: float64

In [53]:
import ipaddress

# Function to map IP to country using the ranges in ip_df
def map_country(ip):
    row = ip_df[(ip_df['lower_bound_ip_address'] <= ip) & (ip_df['upper_bound_ip_address'] >= ip)]
    return row['country'].values[0] if not row.empty else 'Unknown'

# Apply to fraud_df
fraud_df = fraud_df.rename(columns={'ip_address': 'ip_address_int'})
fraud_df['country'] = fraud_df['ip_address_int'].apply(map_country)

In [54]:
print(fraud_df['country'].value_counts().head(10))
print(fraud_df.columns.tolist())


country
United States        58049
Unknown              21966
China                12038
Japan                 7306
United Kingdom        4490
Korea Republic of     4162
Germany               3646
France                3161
Canada                2975
Brazil                2961
Name: count, dtype: int64
['user_id', 'signup_time', 'purchase_time', 'purchase_value', 'device_id', 'source', 'browser', 'sex', 'age', 'ip_address_int', 'class', 'country']


In [41]:
# Transactions per user
user_tx_count = fraud_df.groupby('user_id').size().reset_index(name='transaction_count')
fraud_df = fraud_df.merge(user_tx_count, on='user_id', how='left')

In [58]:
# time based feature
fraud_df['hour_of_day'] = fraud_df['purchase_time'].dt.hour
fraud_df['day_of_week'] = fraud_df['purchase_time'].dt.dayofweek
fraud_df['time_since_signup'] = (fraud_df['purchase_time'] - fraud_df['signup_time']).dt.total_seconds() / 3600  # in hours
print(fraud_df.head())
print(fraud_df.columns.tolist())

   user_id         signup_time       purchase_time  purchase_value  \
0    22058 2015-02-24 22:55:49 2015-04-18 02:47:11              34   
1   333320 2015-06-07 20:39:50 2015-06-08 01:38:54              16   
2     1359 2015-01-01 18:52:44 2015-01-01 18:52:45              15   
3   150084 2015-04-28 21:13:25 2015-05-04 13:54:50              44   
4   221365 2015-07-21 07:09:52 2015-09-09 18:40:53              39   

       device_id source browser sex  age  ip_address_int  class  \
0  QVPSPJUOCKZAR    SEO  Chrome   M   39    7.327584e+08      0   
1  EOGFQPIZPYXFZ    Ads  Chrome   F   53    3.503114e+08      0   
2  YSSKYOSJHPPLJ    SEO   Opera   M   53    2.621474e+09      1   
3  ATGTXKYKUDUQN    SEO  Safari   M   41    3.840542e+09      0   
4  NAUITBZFJKHWW    Ads  Safari   M   45    4.155831e+08      0   

         country  hour_of_day  day_of_week  time_since_signup  
0          Japan            2            5        1251.856111  
1  United States            1            0      

In [59]:
from sklearn.model_selection import train_test_split

fraud_cleaned = fraud_df.drop(['signup_time', 'purchase_time', 'ip_address_int'], axis=1)

X = fraud_cleaned.drop(['class'], axis=1)
y = fraud_cleaned['class']
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

non_numeric = X_train.select_dtypes(include=['object', 'string'])
print("Non-numeric columns:", non_numeric.columns.tolist())
print("Sample values:")
print(non_numeric.head())

Non-numeric columns: ['device_id', 'source', 'browser', 'country']
Sample values:
            device_id  source  browser  country
50481   XVRUVPTHWQASG     Ads   Chrome    China
95673   YEIHKJKVXOTOG  Direct       IE  Unknown
139063  TUIGQGWCACDLN     Ads       IE  Unknown
28313   DSHXPRGHOCLEK  Direct  FireFox    China
123477  NYFGHBQGURDIN     SEO   Safari  Unknown


In [70]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import LabelEncoder
import numpy as np


# 1. Drop columns we don't need or already processed elsewhere
fraud_cleaned = fraud_df.drop(['signup_time', 'purchase_time', 'ip_address_int'], axis=1)

# 2. Frequency encode high-cardinality 'device_id'
device_freq = fraud_cleaned['device_id'].value_counts().to_dict()
fraud_cleaned['device_id_freq'] = fraud_cleaned['device_id'].map(device_freq)
fraud_cleaned.drop('device_id', axis=1, inplace=True)

# 3. One-hot encode low-cardinality categorical columns
fraud_cleaned = pd.get_dummies(fraud_cleaned, columns=['browser', 'source', 'country'], drop_first=True)

# 4. Final check for non-numeric columns
non_numeric_cols = fraud_cleaned.select_dtypes(include=['object', 'string']).columns
print("Non-numeric columns (should be empty):", non_numeric_cols)

# 5. Separate features and target
X = fraud_cleaned.drop('class', axis=1)
y = fraud_cleaned['class']

# 6. Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)


label_encoders = {}
for col in X_train.select_dtypes(include=['object', 'category']).columns:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])
    label_encoders[col] = le

X_train = X_train.astype(np.float32)  


# 7. Apply SMOTE
sm = SMOTE(random_state=42)
X_resampled, y_resampled = sm.fit_resample(X_train, y_train)

print("Remaining non-numeric columns:", X_train.select_dtypes(include=['object', 'category']).columns.tolist())


# 8. Scale features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)



Non-numeric columns (should be empty): Index([], dtype='object')
Remaining non-numeric columns: []


In [ ]:
plt.figure(figsize=(14, 10))

# Select only numeric columns before correlation
numeric_df = fraud_df.select_dtypes(include=['number'])

corr = numeric_df.corr()

sns.heatmap(corr, cmap='coolwarm', annot=False, fmt=".2f", linewidths=0.5)
plt.title("Correlation Heatmap for Ecommerce Fraud Dataset")
plt.show()


In [74]:
# Save cleaned ecommerce fraud data
fraud_df.to_csv('../data/cleaned/cleaned_fraud_df.csv', index=False)

# Save cleaned credit card fraud data
credit_df.to_csv('../data/cleaned/cleaned_creditcard_df.csv', index=False)